<h1 style="text-align:center;font-size:36px">NOTEBOOK DE BAC A SABLE</h1>

In [117]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import geopandas as gp
import numpy as np
from shapely.geometry import Point
from shapely.ops import transform
import pyproj
import re, os
import Connexion_Transfert as ct
import Import_trafics as it
import Donnees_horaires as dh
import Donnees_sources as ds
import Outils as O
from Params import Bdd_OTV as pb

pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 300)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# donnees Tipi

In [ ]:
donneesMinutes = pd.read_csv(r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\tipi_alienor_raw.v2.csv', delimiter=',')

In [ ]:
# liste des stations TIPI
listIdCompteurTipi=donneesMinutes.PME_ID.unique()
# liste des stations OTV
with ct.ConnexionBdd('local_otv_boulot') as c: 
    listObsSupl  =pd.read_sql("select distinct id_comptag, id_cpt, obs_supl from comptage.compteur where gestionnai='DIRA'", c.sqlAlchemyConn)
listStationOtvBrut = listObsSupl.obs_supl.apply(lambda x : x.split(';')[1].split('station : ')[1] if not pd.isnull(x) and 'EMC' not in x and ';' in x else x).unique()
listStationOtvBrut2 = [e.split(',') for e in listStationOtvBrut if e and re.search('^M.*\.._.*$', e)]
listStationOtv = [a.strip().replace('_','') for b in listStationOtvBrut2 for a in b]

In [ ]:
# station TIPI non présentes dans OTV : 
StationTipiHorsOTV = [s for s in listIdCompteurTipi if s not in listStationOtv]
StationOtvHorsTipi = [s for s in listStationOtv if s not in listIdCompteurTipi]
StationOtvTipi = [s for s in listIdCompteurTipi if s in listStationOtv]

# TEST OUVERTUR FICHIERE TAGMASTER .DAT

# donnees CD16 comptages temporaires

In [ ]:
cd16Temp=ds.FIM(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\en_cours\CD16_Temporaires\14-5.fim', gest='CD16')

In [ ]:
cd16Temp.geoloc, cd16Temp.geom_l93.wkt

In [ ]:
geom_l93

In [ ]:
# parcourir un dossier, vérifier si le le FIM est geoloc et récupérer la geom si c'est le cas
dossierCptTemp=r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\en_cours\CD16_Temporaires'
dicoDf={'fichier': [], 'geom': [], 'tmja': [], 'pc_pl': []}
with os.scandir(dossierCptTemp) as it:
    for e in it:
        if e.is_file() and e.name.lower().endswith('.fim'):
            cpt=ds.FIM(e, verifQualite='Message')
            if cpt.geoloc:
                dicoDf['fichier'].append(e.name)
                dicoDf['geom'].append(cpt.geom_l93)
                dicoDf['tmja'].append(cpt.tmja)
                dicoDf['pc_pl'].append(cpt.tmja)

In [ ]:
df=pd.DataFrame(dicoDf)
gdf=gp.GeoDataFrame(df, geometry=df.geom, crs='epsg:2154')
gdf.drop('geom', axis=1).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\testGeomTemp.shp')

In [ ]:
cpt.geom_l93.wkt

In [ ]:
wgs84Proj = pyproj.CRS('EPSG:4326')
l93Proj = pyproj.CRS('EPSG:2154')
project = pyproj.Transformer.from_crs(wgs84Proj, l93Proj).transform
geoloc=re.search('(?P<lat>(\-|\+)[0-9]{1,3}\.([0-9]{4}\.){2})(?P<long>(\-|\+)[0-9]{1,3}\.([0-9]{4}\.){2})', '+45.6021.6666.-000.2134.6333.')
longitude = geoloc.group('long')
latitude = geoloc.group('lat')
x_wgs84=float(re.sub('(\+|-)(.*\.)(.*)(\.)(.*)(\.)', '\g<1>\g<2>\g<3>\g<5>', longitude))
y_wgs84=float(re.sub('(\+|-)(.*\.)(.*)(\.)(.*)(\.)', '\g<1>\g<2>\g<3>\g<5>', latitude))
geom_wgs84=Point(y_wgs84, x_wgs84 )
geom_l93 = transform(project, geom_wgs84)

In [ ]:
geom_l93.wkt

In [ ]:
cpt.dfHoraire2Sens.pivot(index=['date', 'type_veh', 'fichier'], columns='heure', values='nbVeh').reset_index().rename(
    columns={k: v for k, v in zip(['date', 'type_veh']+[e for e in range(24)], ['jour', 'indicateur']+dh.attributsHoraire)})

In [ ]:
cpt.dfHoraire2Sens

# test donnees 6 minutes vikings

In [ ]:
vik = ds.Viking(r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\P5_.VIK')

In [ ]:
dfGroupe=ds.GroupeCompletude(ds.NettoyageTemps(vik.dfFichier), '6T', vitesse=True)#.iloc[:100]

In [ ]:
vik.dfFichier

In [ ]:
dfGroupe.to_csv(r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\P5_.csv')

In [ ]:
dfValide = ds.NettoyageTemps(vik.dfFichier)
frequence='6T'

In [ ]:
dfGroupTypeHeure=dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'type_veh','sens'])['nbVeh'].count().reset_index().sort_values('date_heure')
#completude des données
#ajout des données horaires à 0 si aucun type de vehicules mesures
date_range=pd.date_range(dfGroupTypeHeure.date_heure.min(),dfGroupTypeHeure.date_heure.max(),freq=frequence)
# df de comparaison
dfComp=pd.DataFrame({'type_veh':['2R','PL','VL']}).assign(key=1).merge(pd.DataFrame({'date_heure':date_range}).assign(key=1), on='key').merge(
pd.DataFrame({'sens':['sens1','sens2']}).assign(key=1)).sort_values(['date_heure', 'type_veh','sens'])[['date_heure','type_veh','sens']]
# df des données amnquantes
dfManq=dfComp.loc[dfComp.apply(lambda x : (x['date_heure'],x['type_veh'],x['sens']) not in zip(dfGroupTypeHeure.date_heure.tolist(),dfGroupTypeHeure.type_veh.tolist(),
                  dfGroupTypeHeure.sens.tolist()), axis=1)].copy()
dfHeureTypeSens = pd.concat([dfManq,dfGroupTypeHeure],axis=0).fillna(0).sort_values(['date_heure', 'type_veh','sens'])
dfHeureTypeSens['jour']=dfHeureTypeSens.date_heure.dt.dayofweek
dfHeureTypeSens['jourAnnee']=dfHeureTypeSens.date_heure.dt.dayofyear
dfHeureTypeSens['heure']=dfHeureTypeSens.date_heure.dt.hour
dfHeureTypeSens['date']=pd.to_datetime(dfHeureTypeSens.date_heure.dt.date)
dfHeureTypeSens=pd.concat([dfHeureTypeSens.groupby(['date_heure','sens']).agg({'nbVeh':'sum','heure':lambda x : x.unique()[0], 
                    'jour':lambda x : x.unique()[0],'jourAnnee':lambda x : x.unique()[0],'date':lambda x : x.unique()[0]}
                    ).assign(type_veh='TV').reset_index(),dfHeureTypeSens], axis=0, sort=False).sort_values(['date_heure', 'type_veh'])

In [ ]:
dfGroupVtsHeure=pd.concat([dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure').assign(type_veh='TV'),
           dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens', 'type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure')])
dfHeureTypeSensV=dfHeureTypeSens.merge(dfGroupVtsHeure, on=['date_heure','sens', 'type_veh'],how='left').sort_values(['date_heure', 'type_veh', 'sens'])

In [ ]:
dfHeureTypeSensV

In [ ]:
pd.concat([dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure').assign(type_veh='TV'),
           dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens', 'type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure')])

# affectation auto des points de comptage
> on part des points de comptages nouveau avec une géométrie (exemple dans le 47)<br>
***EN COURS<BR>CHAPITRE 6.2.2 ET 2CRITURE DE LA FONCTION creerComptageAssoc(). L'IDEE EST DE CREER LES DF DIRECTEMENT TELEVERSABLES DANS LA BDD.<br>ENSUITE IL FAUDRA VERIFIER QUE TOUTE LES ETAPES DU SCHEMA [geolocaliserPtComptage](C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\travail\methode\Schemas_OTV.mdj) SONT BIEN REALISEE. JE PENSE QUE LE PB A L'EXECUTION DE LA FONCTION VIENT DE LA FONCTIO MERGE QUI CREE DES _x ET _y SUR DES ATTRIBUTS QUE JE SOUHAITE REFERENCER ENSUITE***


In [77]:
# recupérer les données
# params : nom de table  ou df des points geolocalises, nom de table de linauto, nom de Bdd
with ct.ConnexionBdd('local_otv_boulot') as c:
    pt47 = gp.read_postgis('select * from cd47_2020', c.sqlAlchemyConn)
    pt47Geom = gp.read_postgis('select * from cd47_2020 where geom is not null', c.sqlAlchemyConn)
    cptSansGeom = gp.read_postgis('select * from cd47_2020 where geom is null', c.sqlAlchemyConn)
    linauto47 = gp.read_postgis("""
                                 select DISTINCT ON (t.gid) c.id_comptag id_comptag_bdd,c.type_poste type_poste_bdd, t.gid, t.geom, st_distance(c.geom, t.geom)
                                  from (SELECT * FROM linauto.traf2020_bdt_na_ed20_simpli_l where list_dept like '%%47%%') t LEFT JOIN comptage.compteur c ON st_dwithin(c.geom, t.geom, 30)
                                  ORDER BY t.gid, CASE WHEN c.type_poste = 'permanent' THEN 1 
                                                       WHEN c.type_poste = 'tournant' THEN 2
                                                       WHEN c.type_poste = 'ponctuel' THEN 3 
                                                       ELSE 4 
                                                       end
                                 """, c.sqlAlchemyConn)

## Prealable
> ici on va preparer les donnes et fonctions generiques

### preparation des donnees

In [78]:
# on en déduit le plus proche voisin entre les points et la linauto
ppV = O.plus_proche_voisin(pt47Geom, linauto47, 10, 'id_comptag', 'gid')
ppVSectHomo = ppV.loc[~ppV.gid.isna()].copy()
ppvHorsSectHomo = ppV.loc[ppV.gid.isna()].copy()  # PENSER A CREER LES COMPTEURS POUR CEUX LA !!!!!
# et on récupère l'id_comptag_existant et l'id de section homogene
ppVTot = pt47Geom.merge(ppVSectHomo, how='left', on='id_comptag').merge(linauto47, how='left', on='gid')

# trouver le nombre de comptage par section homogène (gid)
nbCptSectHomo = ppVTot.gid.value_counts()

# separer les differents cas a traiter
cptSimpleSectHomo = ppVTot.loc[ppVTot.gid.isin(nbCptSectHomo.loc[nbCptSectHomo == 1].index.tolist())].copy()
cptMultiSectHomo = ppVTot.loc[ppVTot.gid.isin(nbCptSectHomo.loc[nbCptSectHomo > 1].index.tolist())].copy()

### Fonctions génériques

In [79]:
def creerCompteur(cptRef, attrGeom, dep, reseau, gestionnai, concession, techno=None, src_geo=None, obs_geo=None, obs_supl=None, id_cpt=None,
                  id_sect=None, fictif=False, en_service=True):
    """
    creation d'une df prete a etre integree dans la table des compteurs
    presque tout les chmaps 
    in : 
        cptRef : ou geodataframe avec un attribut de géométrie en SRID = 2154
        attrGeom : string : nom de l'attribut supportant la géométrie
        dep : departement sur 2 caractère ou pd.Series
        reseau : valeur depuis liste enumeree (cf bdd) ou pd.Series de ces valeurs
        gestionnai : valeur depuis liste enumeree (cf bdd) ou pd.Series de ces valeurs
        concession : boolean ou pd.Series de booleean
    """
    df = cptRef.copy()
    O.checkAttributsinDf(df, [attrGeom, 'id_comptag', 'type_poste', 'periode', 'pr', 'absc', 'route', 'src_cpt', 'convention', 'sens_cpt'])
    df = df[[attrGeom, 'id_comptag', 'type_poste', 'periode', 'pr', 'absc', 'route', 'src_cpt', 'convention', 'sens_cpt']].assign(
        dep=dep, reseau=reseau, gestionnai=gestionnai, concession=concession, techno=techno, src_geo=src_geo, obs_geo=obs_geo,
        obs_supl=obs_supl, id_cpt=id_cpt, id_sect=id_sect, fictif=fictif, en_service=en_service)
    df['x_l93'] = df[attrGeom].apply(lambda x: round(x.x,3))
    df['y_l93'] = df[attrGeom].apply(lambda x: round(x.y,3))
    df.drop(
        [c for c in df.columns if c not in ('dep', 'reseau', 'gestionnai', 'concession', attrGeom,
                                            'id_comptag', 'type_poste', 'periode', 'pr', 'absc', 'route', 'src_cpt', 'convention', 'sens_cpt', 'reseau', 'dep',
                                            'reseau', 'gestionnai', 'concession', 'techno', 'src_geo', 'obs_geo', 'obs_supl', 'id_cpt', 'id_sect', 'fictif',
                                            'en_service')], axis=1, inplace=True)
    gdfCpt = gp.GeoDataFrame(df, geometry=attrGeom, crs=2154)
    gdfCpt = O.gp_changer_nom_geom(gdfCpt, 'geom')
    return gdfCpt

In [151]:
def rangBddComptageAssoc(id_comptag_ref_Bdd, bdd='local_otv_boulot'):
    """
    Récupérer dans la bdd, le rang max associé à un comptage de réference dans la table comptage_assoc.comptage
    """
    with ct.ConnexionBdd(bdd) as c:
        rqt = f"""SELECT COALESCE(max(cac.rang),0) rang 
                   FROM comptage_assoc.comptage cac JOIN comptage.comptage c ON c.id = cac.id_cptag_ref 
                   WHERE c.id_comptag='{id_comptag_ref_Bdd}'"""
        rang = pd.read_sql(rqt, c.sqlAlchemyConn).iloc[0]
    return rang


# ################# #!!!!! EN COURS !!!!!##############################################################
def creerComptageAssoc(df, id_comptag_ref_nom, annee, id_compteur_asso_nom=None, src=None):
    """
    a partir d'une df, creer la df a injecter dans les tables compteur et comptage du schema comptage_assoc de la bdd
    in : 
        df : la df de base, doit contenir obligatoirement les attributs decrivant l'id_comptag_ref, le rang, le type_veh. si possible la periode, src, obs.
             si le comptage_assoc comprend une dimension géométrique, les attributs decrivant l'id_compteur_asso, le type_poste, la src_geo, la src_cpt, convention,
             et sens_cpt sont obligatoire. si possible ajouter route, pr, abs, techno, obs_geo, obs_supl
        id_comptag_ref_nom : string : nom de l'attribut supportant l'id_comptag du point de référence
        id_compteur_asso_nom : string : nom de l'attribut supportant l'id_comptag du point associé si composante geometrique
        annee : string annee sur 4 caractères
    """
    O.checkAttributsinDf(df, ['type_veh', 'periode'])
    if id_compteur_asso_nom:
        O.checkAttributsinDf(df, ['type_poste', 'src_geo', 'src_cpt', 'convention', 'sens_cpt'])
    instanceCpt = it.Comptage('fake')
    corresIdComptag = instanceCpt.recupererIdUniqComptage(df[[id_comptag_ref_nom]].rename(columns={id_comptag_ref_nom: 'id_comptag'}).assign(annee=annee))
    dfIds = df.merge(corresIdComptag, left_on=id_comptag_ref_nom, right_on='id_comptag', how='left').rename(columns={'id_comptag_uniq': 'id_comptag_ref'}).assign(annee=annee)
    dfIds['rang_bdd'] = dfIds[id_comptag_ref_nom].apply(lambda x: rangBddComptageAssoc(x))
    dfIds['rang_df'] = dfIds.groupby(id_comptag_ref_nom).cumcount()+1
    dfIds['rang'] = dfIds['rang_bdd']+dfIds['rang_df']
    tableComptage = dfIds.rename(columns={id_compteur_asso_nom: 'id_compteur_asso'})[[c for c in pb.attrComptageAssoc if c in dfIds.columns]].copy()
    return dfIds, tableComptage, corresIdComptag
#########################################################################################################

In [99]:
def creerCorrespComptag(df, nomAttrIdComptagGest, nomAttrIdComptagGti, listIdCptExclu):
    """
    creer une df a integrer dans la bdd a partir d'une df comprenant : un attribut id_comptag de la bdd et un attribut id_comptag issu des données gestionnaire.
    si la df commprend des type poste gestionnaire et bdd on peut ajouter le nom de ces attributs pour faire un test 
    in :
        df : df de base doit contenir un attribut id_comptag de la bdd et un attribut id_comptag issu des données gestionnaire
        nomAttrIdComptagGest : string : nom de l'attribut décrivant l'id_comptag issu des données gestionnaire
        nomAttrIdComptagGti : string : nom de l'attribut décrivant l'id_comptag de la bdd
        listIdCptExclu : list ou tuple : list des id_comptage à ne pas prendre en compte
    """
    return df.loc[~df[nomAttrIdComptagGest].isin(listIdCptExclu)][[nomAttrIdComptagGest, nomAttrIdComptagGti]].copy().rename(columns={nomAttrIdComptagGest: 'id_gest', nomAttrIdComptagGti: 'id_gti'})

## CAS 1 : 1 seul comptage sur une section homogène
on va séparer les cas où un id_comptag est déjà présent ou non

In [82]:
cptSimpleSectHomoNew = cptSimpleSectHomo.loc[cptSimpleSectHomo.id_comptag_bdd.isna()].copy()
cptSimpleSectHomoOld = cptSimpleSectHomo.loc[~cptSimpleSectHomo.id_comptag_bdd.isna()].copy()

### Comptage sur section_homogene sans id_comptag
creation des compteurs, comptages et indicateurs

In [ ]:
# analyse visuelle via Qgis
cptSimpleSectHomoNew = gp.GeoDataFrame(cptSimpleSectHomoNew, geometry='geom_x', crs=2154)
cptSimpleSectHomoNew.drop('geom_y', axis=1).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD47\verifCreationCompteur.shp')
# analyse -> ok

In [50]:
# creation des compteurs
attrGeomCompteurNew = 'geom_x'
dep = '47'
reseau = 'RD'
gestionnai = 'CD47'
concession = 'N'
cptSimpleSectHomoNew['src_cpt'] = cptSimpleSectHomoNew.type_poste.apply(lambda x: 'convention_gestionnaire' if x == 'permanent' else 'gestionnaire')
cptSimpleSectHomoNew['convention'] = cptSimpleSectHomoNew.type_poste.apply(lambda x: True if x == 'permanent' else False)
cptSimpleSectHomoNew['sens_cpt'] = 'double sens'
dfCompteurNew = creerCompteur(cptSimpleSectHomoNew, attrGeomCompteurNew, dep, reseau, gestionnai, concession, techno=None, 
                              src_geo=None, obs_geo=None, obs_supl=None, id_cpt=None, id_sect=None, fictif=False, en_service=True)

In [58]:
# creation des comptages (il faut inserer les compteurs avant)
cptFictif = it.Comptage('faux')
cptFictif.creer_comptage(cptSimpleSectHomoNew.id_comptag.tolist(), 2020, 'donnees complete CD47', 'vl/pl', periode=cptSimpleSectHomoNew.periode.tolist())

In [ ]:
# creation des indicateurs (necessite la creation des comptages avant)
cptFictif.structureBddOld2NewForm(cptSimpleSectHomoNew.assign(annee='2020'), '2020', ['id_comptag','annee', 'fichier'], ['tmja', 'pc_pl'], 'agrege')

### Comptage sur section_homogene avec id_comptag
selon la nature du comptage deja présent dans la base et selon la nature du comptage a injecté, on va soit : 
- creer un nouveau comptage associe avec le nouveau point
- creer une correspondance dans corresp_id_comptag
- modifier le type de poste et tracer la modif

In [96]:
def ventilerNouveauComptageRef(df, nomAttrtypePosteGest, nomAttrtypePosteBdd, nomAttrperiode):
    """
    depuis une df des comptage de référence situé sur des tronçons avec un id_comptag existantdans la base, 
    séparer en 3 groupe selon le type de poste dans la bdd et le type de poste du hestionnaire.
    in : 
        df : dataframe a classifier
        nomAttrtypePosteGest : string : nom de l'attribut supportant le type de poste fourni par le gest 
        nomAttrtypePosteBdd : string : nom de l'attribut supportant le type de poste dans la bdd
        nomAttrperiode : string : nom de l'attribut décrivant la période de mesure
    """
    dfCorrespIdComptag = df.loc[((cptSimpleSectHomoOld[nomAttrtypePosteGest] == cptSimpleSectHomoOld[nomAttrtypePosteBdd]) &
                                 (cptSimpleSectHomoOld[nomAttrtypePosteGest] != 'ponctuel')) |
                                ((cptSimpleSectHomoOld[nomAttrtypePosteGest] == cptSimpleSectHomoOld[nomAttrtypePosteBdd]) &
                                 (cptSimpleSectHomoOld[nomAttrtypePosteGest] == 'ponctuel') &
                                 (~cptSimpleSectHomoOld[nomAttrperiode].apply(lambda x: O.verifVacanceRange(x))))].copy()
    dfCreationComptageAssocie = cptSimpleSectHomoOld.loc[((cptSimpleSectHomoOld[nomAttrtypePosteGest] =='ponctuel') & 
                                                          (cptSimpleSectHomoOld[nomAttrtypePosteBdd].isin(('permanent', 'tournant')))) |
                                                         ((cptSimpleSectHomoOld[nomAttrtypePosteGest] == cptSimpleSectHomoOld[nomAttrtypePosteBdd]) &
                                                          (cptSimpleSectHomoOld[nomAttrtypePosteGest] == 'ponctuel') &
                                                          (cptSimpleSectHomoOld[nomAttrperiode].apply(lambda x: O.verifVacanceRange(x))))]
    dfModifTypePoste = df.loc[((df[nomAttrtypePosteGest].isin(('permanent', 'tournant'))) & (df[nomAttrtypePosteBdd] == 'ponctuel')) |
                              ((df[nomAttrtypePosteGest] == 'permanent') & (df[nomAttrtypePosteBdd] == 'tournant'))].copy()
    return dfCorrespIdComptag, dfCreationComptageAssocie, dfModifTypePoste

In [97]:
dfCorrespIdComptag, dfCreationComptageAssocie, dfModifTypePoste = ventilerNouveauComptageRef(cptSimpleSectHomoOld, 'type_poste', 'type_poste_bdd', 'periode')

In [ ]:
# analyse de la table de correspiIdComptag
# export pour vérif
gp.GeoDataFrame(dfCorrespIdComptag.drop('geom_y', axis=1), geometry='geom_x', crs=2154
                ).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD47\verifCorrespIdComptage.shp')

In [102]:
# certains comptages doivent etre "forcé" en nouveau compteur. On fait une liste
listCompteurAForcer = []
listCompteurAForcer.append('47-D211-8+0')

In [ ]:
# analyse de la table de correspiIdComptag
# export pour vérif
gp.GeoDataFrame(dfCreationComptageAssocie.drop('geom_y', axis=1), geometry='geom_x', crs=2154
                ).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD47\verifComptagesAssoc.shp')

In [104]:
# on ajoute les valeurs dans la liste des compteurs a faorcer
listCompteurAForcer.append('47-D124-0+315')
listCompteurAForcer.append('47-D292-2+140')

In [ ]:
# analyse des modifs de type de poste : -> null
dfModifTypePoste

In [107]:
# EN COURS
# EN COURS on cree les correspondances, comptages associes et compteurs (en cours)
correspIdComptag = creerCorrespComptag(dfCorrespIdComptag, 'id_comptag', 'id_comptag_bdd', listCompteurAForcer)

In [153]:
# EN COURS
dfIds, tableComptage, corresIdComptag = creerComptageAssoc(dfCreationComptageAssocie.assign(type_veh='vl/pl',
                                                    src_geo='pr+abs_gestionnaire',
                                                    convention=lambda x: dfCreationComptageAssocie.type_poste.apply(lambda x: True if x == 'permanent' else False),
                                                    sens_cpt='double sens',
                                                    src_cpt=lambda x: dfCreationComptageAssocie.type_poste.apply(lambda x: 'convention gestionnaire'
                                                                                                                 if x == 'permanent' else 'gestionnaire')),
                   'id_comptag_bdd', '2020', id_compteur_asso_nom='id_comptag', src=None)

In [123]:
# EN COURS
creerComptageAssoc(dfCreationComptageAssocie.assign(type_veh='vl/pl',
                                                    src_geo='pr+abs_gestionnaire',
                                                    convention=lambda x: dfCreationComptageAssocie.type_poste.apply(lambda x: True if x == 'permanent' else False),
                                                    sens_cpt='double sens',
                                                    src_cpt=lambda x: dfCreationComptageAssocie.type_poste.apply(lambda x: 'convention gestionnaire'
                                                                                                                 if x == 'permanent' else 'gestionnaire')),
                   'id_comptag_bdd', '2020', id_compteur_asso_nom='id_comptag', src=None)

KeyError: "['obs', 'id_comptag_ref', 'id_cpteur_asso'] not in index"

## CAS 2 : plusieurs comptages sur une section homogène
Il va falloir déterminer quel est le comptage de référence (cf doc)

pour pouvoir déterminer le comptage de référence il faut connaitre : 
1. les types de postes (priorité : permanent > tournant > ponctuel)
1. les périodes de comptages (permanent non concerné)
1. les vacances scolaires
1. les niveaux de TMJA
1. le niveau de pc_pl
1. un choix arbitraire

### fonction de choix d'un comptage de reference

In [8]:
# creation de la fonction de hiérarchisation
def hierarchisationCompteur(typePoste, periode, tmja, pc_pl):
    """
    obtention d"'une note globale de hierarhisation, a partir des 3 fonction incluse'
    """
    def hierarchisationTypePoste(typePoste):
        """
        attribuer une valeur de 3*10**21, 2*10**21 ou 10**21 selon le type de poste
        in : 
            typePoste : string parmi 'permanent', 'tournant', 'ponctuel'
        """
        O.checkParamValues(typePoste, ('permanent', 'tournant', 'ponctuel'))
        if typePoste == 'permanent':
            return 3*10**15
        elif typePoste == 'tournant':
            return 2*10**15
        else:
            return 10**15

    def hierarchisationVacance(periode):
        """
        attribuer une valeur de 2*10**18 ou 10**18 selon qu'une partie de la mesure a été faie pendant les vacances scolaire
        in :
            periode : string sou forme YYYY/MM/DD-YYYY/MM/DD éventuellement séparé par des ' ; '
        """
        testVacances = O.verifVacanceRange(periode)
        if not periode or testVacances:
            return 10**12
        else:
            return 2*10**12

    def hierarchisationTrafic(tmja, pc_pl):
        """
        fournir une valeur qui concatene le trafic et le pc_pl, en favorisant le trafic
        """
        if not tmja or tmja <= 0: 
            return ValueError('le tmja doit etre une superieur a 0  non nulle')
        elif not pc_pl:
            return tmja * 1000
        else:
            return (tmja * 1000) + pc_pl

    return hierarchisationTypePoste(typePoste) + hierarchisationVacance(periode) + hierarchisationTrafic(tmja, pc_pl)

In [9]:
# calcul des compteur de reference et associes
cptMultiSectHomo['note_hierarchise'] = cptMultiSectHomo.apply(lambda x: hierarchisationCompteur(x.type_poste, x.periode, x.tmja, x.pc_pl), axis=1)
cptRefMultiSectHomo = cptMultiSectHomo.loc[cptMultiSectHomo.groupby('gid').note_hierarchise.transform('max') == cptMultiSectHomo.note_hierarchise].sort_values('gid').copy()
cptAssocMultiSectHomo = cptMultiSectHomo.loc[cptMultiSectHomo.groupby('gid').note_hierarchise.transform('max') != cptMultiSectHomo.note_hierarchise].sort_values('gid').copy()
cptAssocMultiSectHomo = cptAssocMultiSectHomo.merge(cptRefMultiSectHomo[['gid', 'id_comptag']], on='gid', suffixes=(None, '_ref'))
# verif que tous les gid ont un cpt ref
if not cptAssocMultiSectHomo.loc[~cptAssocMultiSectHomo.gid.isin(cptRefMultiSectHomo.gid.unique())].empty:
    raise ValueError('un des comptage associe n\'a pas de comptage de reference')
if not len(cptRefMultiSectHomo)+len(cptAssocMultiSectHomo) == len(cptMultiSectHomo):
    raise ValueError('un ou plusieurs comptage n\'ont pas ete affecte comme reference ou associe')

In [10]:
# on sépare les comptages de références selon qu'ils sont sur un id_comptag de la bdd ou non
cptRefMultiSectHomoNew = cptRefMultiSectHomo.loc[cptRefMultiSectHomo.id_comptag_bdd.isna()].copy()
cptRefMultiSectHomoOld = cptRefMultiSectHomo.loc[~cptRefMultiSectHomo.id_comptag_bdd.isna()].copy()

### Comptage de référence sur section_homogene sans id_comptag dans la bdd
creation des compteurs, comptages, indicateurs et association des comptages associés calculé plus haut

In [11]:
# création de variable et mise en forme de la df
cptRefMultiSectHomoNew['src_cpt'] = cptRefMultiSectHomoNew.type_poste.apply(lambda x: 'convention_gestionnaire' if x == 'permanent' else 'gestionnaire')
cptRefMultiSectHomoNew['convention'] = cptRefMultiSectHomoNew.type_poste.apply(lambda x: True if x == 'permanent' else False)
cptRefMultiSectHomoNew['sens_cpt'] = 'double sens'

# creer la gdf des nouveaux compteurs issus de ce cas (avec comme attributs ceux dploye au 6.2.1)
newCompteurMultiSectHomo = creerCompteur(cptRefMultiSectHomoNew, attrGeomcptRefMultiSectHomoNew, dep, reseau, gestionnai, concession, techno=None, src_geo=None, obs_geo=None, obs_supl=None, id_cpt=None,
                                         id_sect=None, fictif=False, en_service=True)

### Comptage de référence sur section_homogene avec id_comptag
selon la nature du comptage deja présent dans la base et selon la nature du comptage a injecté, on va soit : 
- creer un nouveau comptage associe avec le nouveau point
- creer une correspondance dans corresp_id_comptag
- modifier le type de poste et tracer la modif

In [25]:
dfCorrespIdComptag, dfCreationComptageAssocie, dfModifTypePoste = ventilerNouveauComptageRef(cptRefMultiSectHomoOld, 'type_poste', 'type_poste_bdd')

Pour chaque donnes cree ci-dessus, il faut vérifier les associations, pour pouvoir lister les comptages mal défini 
exemple : 
- comptage pour lesquels on force la création d'un compteur
- erreur du a une erreur de lineairisation

In [26]:
# analyse de la table de correspiIdComptag
correspIdComptag = creerCorrespComptag(dfCorrespIdComptag, 'id_comptag', 'id_comptag_bdd')
creationForceeCompteur = ['47-D276-10+905']

In [27]:
# ananlyse de la table dfCreationComptageAssocie : correction d'une coquille due à la linauto
dfCreationComptageAssocie.drop(34, inplace=True)

In [ ]:
# analyse de la table dfCreationComptageAssocie
dfModifTypePoste